In [1]:

import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from math import sqrt

In [2]:

diabetes=pd.read_csv('/content/diabetes.csv')

diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:

diabetes.loc[diabetes['Glucose'] == 0,'Glucose'] = np.nan
diabetes.loc[diabetes['BloodPressure'] == 0,'BloodPressure'] = np.nan
diabetes.loc[diabetes['SkinThickness'] == 0,'SkinThickness'] = np.nan
diabetes.loc[diabetes['Insulin'] == 0,'Insulin'] = np.nan
diabetes.loc[diabetes['BMI'] == 0,'BMI'] = np.nan
diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,NaN,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,NaN,NaN,30.1,0.349,47,1


In [4]:

diabetes.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [5]:

(diabetes.Glucose.isnull().sum())/ len('diabetes')

0.625

In [6]:

(diabetes.BloodPressure.isnull().sum())/ len('diabetes')

4.375

In [7]:

(diabetes.SkinThickness.isnull().sum())/ len('diabetes')

28.375

In [8]:

(diabetes.Insulin.isnull().sum())/ len('diabetes')

46.75

In [9]:

(diabetes.BMI.isnull().sum())/ len('diabetes')

1.375

In [10]:

diabetes.drop(["SkinThickness","Insulin"],axis=1,inplace=True)



In [11]:
diabetes_missing_value_threshold=diabetes
diabetes_missing_value_threshold

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,33.6,0.627,50,1
1,1,85.0,66.0,26.6,0.351,31,0
2,8,183.0,64.0,23.3,0.672,32,1
3,1,89.0,66.0,28.1,0.167,21,0
4,0,137.0,40.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...
763,10,101.0,76.0,32.9,0.171,63,0
764,2,122.0,70.0,36.8,0.340,27,0
765,5,121.0,72.0,26.2,0.245,30,0
766,1,126.0,60.0,30.1,0.349,47,1


In [12]:
diabetes_missing_value_threshold_features = diabetes_missing_value_threshold.drop('Outcome',axis=1)
diabetes_missing_value_threshold_label= diabetes_missing_value_threshold['Outcome']
diabetes_missing_value_threshold_features,diabetes_missing_value_threshold_label


(     Pregnancies  Glucose  BloodPressure   BMI  DiabetesPedigreeFunction  Age
 0              6    148.0           72.0  33.6                     0.627   50
 1              1     85.0           66.0  26.6                     0.351   31
 2              8    183.0           64.0  23.3                     0.672   32
 3              1     89.0           66.0  28.1                     0.167   21
 4              0    137.0           40.0  43.1                     2.288   33
 ..           ...      ...            ...   ...                       ...  ...
 763           10    101.0           76.0  32.9                     0.171   63
 764            2    122.0           70.0  36.8                     0.340   27
 765            5    121.0           72.0  26.2                     0.245   30
 766            1    126.0           60.0  30.1                     0.349   47
 767            1     93.0           70.0  30.4                     0.315   23
 
 [768 rows x 6 columns], 0      1
 1      0
 2    

In [13]:

diabetes_missing_value_threshold_features.head()

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,33.6,0.627,50
1,1,85.0,66.0,26.6,0.351,31
2,8,183.0,64.0,23.3,0.672,32
3,1,89.0,66.0,28.1,0.167,21
4,0,137.0,40.0,43.1,2.288,33


In [14]:

diabetes_missing_value_threshold_label.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [15]:


diabetes_clean= pd.read_csv("/content/diabetes_cleaned.csv")
diabetes_clean.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,218.937760,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,70.189298,26.6,0.351,31.0,0
2,8.0,183.0,64.0,29.0,269.968908,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94.000000,28.1,0.167,21.0,0
4,0.0,137.0,40.0,35.0,168.000000,43.1,2.288,33.0,1


In [16]:

X=diabetes_clean.drop('Outcome',axis=1)
Y=diabetes_clean['Outcome']


In [17]:

X.var(axis=0)

Pregnancies                   11.354056
Glucose                      929.680350
BloodPressure                146.321591
SkinThickness                 77.285567
Insulin                     9484.259268
BMI                           48.813618
DiabetesPedigreeFunction       0.109779
Age                          138.303046
dtype: float64

In [18]:

from sklearn.preprocessing import minmax_scale
X_scaled_df =pd.DataFrame(minmax_scale(X,feature_range=(0,10)),columns=X.columns)



In [19]:

X_scaled_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3.529412,6.709677,4.897959,3.043478,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,2.391304,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,2.391304,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.739130,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,3.043478,2.150572,5.092025,9.436379,2.000000
...,...,...,...,...,...,...,...,...
763,5.882353,3.677419,5.306122,4.456522,2.289500,3.006135,0.397096,7.000000
764,1.176471,5.032258,4.693878,2.173913,2.044244,3.803681,1.118702,1.000000
765,2.941176,4.967742,4.897959,1.739130,1.502241,1.635992,0.713066,1.500000
766,0.588235,5.290323,3.673469,2.391304,2.217956,2.433538,1.157131,4.333333


In [20]:


X_scaled_df.var()

Pregnancies                 3.928739
Glucose                     3.869637
BloodPressure               1.523548
SkinThickness               0.913109
Insulin                     1.271218
BMI                         2.041377
DiabetesPedigreeFunction    2.001447
Age                         3.841751
dtype: float64

In [21]:

from sklearn.feature_selection import VarianceThreshold

select_features = VarianceThreshold(threshold=1.0)




In [22]:


X_variance_threshold_df=select_features.fit_transform(X_scaled_df)
X_variance_threshold_df

array([[3.52941176, 6.70967742, 4.89795918, ..., 3.14928425, 2.3441503 ,
        4.83333333],
       [0.58823529, 2.64516129, 4.28571429, ..., 1.71779141, 1.16567037,
        1.66666667],
       [4.70588235, 8.96774194, 4.08163265, ..., 1.04294479, 2.53629377,
        1.83333333],
       ...,
       [2.94117647, 4.96774194, 4.89795918, ..., 1.63599182, 0.71306576,
        1.5       ],
       [0.58823529, 5.29032258, 3.67346939, ..., 2.43353783, 1.15713066,
        4.33333333],
       [0.58823529, 3.16129032, 4.69387755, ..., 2.49488753, 1.01195559,
        0.33333333]])

In [23]:

X_variance_threshold_df

array([[3.52941176, 6.70967742, 4.89795918, ..., 3.14928425, 2.3441503 ,
        4.83333333],
       [0.58823529, 2.64516129, 4.28571429, ..., 1.71779141, 1.16567037,
        1.66666667],
       [4.70588235, 8.96774194, 4.08163265, ..., 1.04294479, 2.53629377,
        1.83333333],
       ...,
       [2.94117647, 4.96774194, 4.89795918, ..., 1.63599182, 0.71306576,
        1.5       ],
       [0.58823529, 5.29032258, 3.67346939, ..., 2.43353783, 1.15713066,
        4.33333333],
       [0.58823529, 3.16129032, 4.69387755, ..., 2.49488753, 1.01195559,
        0.33333333]])

In [24]:

X_variance_threshold_df=pd.DataFrame(X_variance_threshold_df)

In [25]:
 
X_variance_threshold_df.head()

,0,1,2,3,4,5,6
0,3.529412,6.709677,4.897959,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,2.150572,5.092025,9.436379,2.000000


In [26]:
def get_selected_features(raw_df,processed_df):
    selected_features=[]
    for i in range(len(processed_df.columns)):
        for j in range(len(raw_df.columns)):
            if (processed_df.iloc[:,i].equals(raw_df.iloc[:,j])):
                selected_features.append(raw_df.columns[j])
    return selected_features

In [27]:


selected_features= get_selected_features(X_scaled_df,X_variance_threshold_df)
selected_features



['Pregnancies',
 'Glucose',
 'BloodPressure',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [28]:

X_variance_threshold_df.columns=selected_features
selected_features_df = X_variance_threshold_df
selected_features_df



,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3.529412,6.709677,4.897959,2.740295,3.149284,2.344150,4.833333
1,0.588235,2.645161,4.285714,1.018185,1.717791,1.165670,1.666667
2,4.705882,8.967742,4.081633,3.331099,1.042945,2.536294,1.833333
3,0.588235,2.903226,4.285714,1.293850,2.024540,0.380017,0.000000
4,0.000000,6.000000,1.632653,2.150572,5.092025,9.436379,2.000000
...,...,...,...,...,...,...,...
763,5.882353,3.677419,5.306122,2.289500,3.006135,0.397096,7.000000
764,1.176471,5.032258,4.693878,2.044244,3.803681,1.118702,1.000000
765,2.941176,4.967742,4.897959,1.502241,1.635992,0.713066,1.500000
766,0.588235,5.290323,3.673469,2.217956,2.433538,1.157131,4.333333


In [29]:
def generate_feature_scores_df(X,Score):
    feature_score=pd.DataFrame()
    for i in range(X.shape[1]):
        new =pd.DataFrame({"Features":X.columns[i],"Score":Score[i]},index=[i])
        feature_score=pd.concat([feature_score,new])
    return feature_score

In [30]:

diabetes=pd.read_csv('/content/diabetes.csv')

In [31]:

X=diabetes.drop('Outcome',axis=1)
Y=diabetes['Outcome']


In [32]:

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest


In [33]:

X=X.astype(np.float64)

In [34]:


chi2_test=SelectKBest(score_func=chi2,k=4)
chi2_model=chi2_test.fit(X,Y)





In [35]:

chi2_model.scores_

array([ 111.51969064, 1411.88704064,   17.60537322,   53.10803984,
       2175.56527292,  127.66934333,    5.39268155,  181.30368904])

In [36]:


feature_score_df=generate_feature_scores_df(X,chi2_model.scores_)
feature_score_df




,Features,Score
0,Pregnancies,111.519691
1,Glucose,1411.887041
2,BloodPressure,17.605373
3,SkinThickness,53.108040
4,Insulin,2175.565273
5,BMI,127.669343
6,DiabetesPedigreeFunction,5.392682
7,Age,181.303689


In [37]:

X_new=chi2_model.transform(X)

In [38]:

X_new=pd.DataFrame(X_new)


In [39]:

selected_features=get_selected_features(X,X_new)
selected_features




['Glucose', 'Insulin', 'BMI', 'Age']

In [40]:
chi2_best_features=X[selected_features]
chi2_best_features.head()



,Glucose,Insulin,BMI,Age
0,148.0,0.0,33.6,50.0
1,85.0,0.0,26.6,31.0
2,183.0,0.0,23.3,32.0
3,89.0,94.0,28.1,21.0
4,137.0,168.0,43.1,33.0


In [41]:


from sklearn.feature_selection import f_classif,SelectPercentile
Anova_test=SelectPercentile(f_classif,percentile=80)
Anova_model= Anova_test.fit(X,Y)



In [42]:

Anova_model.scores_

array([ 39.67022739, 213.16175218,   3.2569504 ,   4.30438091,
        13.28110753,  71.7720721 ,  23.8713002 ,  46.14061124])

In [43]:
 
feature_scores_df=generate_feature_scores_df(X,Anova_model.scores_)
feature_scores_df



,Features,Score
0,Pregnancies,39.670227
1,Glucose,213.161752
2,BloodPressure,3.256950
3,SkinThickness,4.304381
4,Insulin,13.281108
5,BMI,71.772072
6,DiabetesPedigreeFunction,23.871300
7,Age,46.140611


In [44]:

X_new=Anova_model.transform(X)




In [45]:
X_new=pd.DataFrame(X_new)

In [46]:

selected_features=get_selected_features(X,X_new)

selected_features

['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

In [47]:

Anova_selected_feature_df=X[selected_features]
Anova_selected_feature_df.head()

,Pregnancies,Glucose,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6.0,148.0,0.0,33.6,0.627,50.0
1,1.0,85.0,0.0,26.6,0.351,31.0
2,8.0,183.0,0.0,23.3,0.672,32.0
3,1.0,89.0,94.0,28.1,0.167,21.0
4,0.0,137.0,168.0,43.1,2.288,33.0


In [48]:

diabetes = pd.read_csv('/content/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [49]:

X= diabetes.drop('Outcome', axis = 1)
Y= diabetes["Outcome"]

print(X)
print(Y)


     Pregnancies  Glucose  BloodPressure  ...   BMI  DiabetesPedigreeFunction  Age
0              6      148             72  ...  33.6                     0.627   50
1              1       85             66  ...  26.6                     0.351   31
2              8      183             64  ...  23.3                     0.672   32
3              1       89             66  ...  28.1                     0.167   21
4              0      137             40  ...  43.1                     2.288   33
..           ...      ...            ...  ...   ...                       ...  ...
763           10      101             76  ...  32.9                     0.171   63
764            2      122             70  ...  36.8                     0.340   27
765            5      121             72  ...  26.2                     0.245   30
766            1      126             60  ...  30.1                     0.349   47
767            1       93             70  ...  30.4                     0.315   23

[76

In [50]:

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


In [51]:

model=LogisticRegression(solver='liblinear')


rfe= RFE(model, n_features_to_select=4)

In [52]:


fit= rfe.fit(X, Y)



In [53]:
print('Number of selected features',fit.n_features_)
print('Selected Features',fit.support_)
print('Feature rankings',fit.ranking_)

Number of selected features 4
Selected Features [ True  True False False False  True  True False]
Feature rankings [1 1 2 4 5 1 1 3]


In [54]:

def feature_ranks(X,Rank,Support):
    feature_rank=pd.DataFrame()
    for i in range(X.shape[1]):
        new =pd.DataFrame({"Features":X.columns[i],"Rank":Rank[i],'Selected':Support[i]},index=[i])
        feature_rank=pd.concat([feature_rank,new])
    return feature_rank


In [55]:

feature_rank_df= feature_ranks(X, fit.ranking_, fit.support_)

feature_rank_df

,Features,Rank,Selected
0,Pregnancies,1,True
1,Glucose,1,True
2,BloodPressure,2,False
3,SkinThickness,4,False
4,Insulin,5,False
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True
7,Age,3,False


In [56]:

recursive_feature_names=feature_rank_df[feature_rank_df['Rank'] == 1]

recursive_feature_names


,Features,Rank,Selected
0,Pregnancies,1,True
1,Glucose,1,True
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True


In [57]:

col = X.columns[rfe.support_]
RFE_selected_features=X[col]


RFE_selected_features.head()


,Pregnancies,Glucose,BMI,DiabetesPedigreeFunction
0,6,148,33.6,0.627
1,1,85,26.6,0.351
2,8,183,23.3,0.672
3,1,89,28.1,0.167
4,0,137,43.1,2.288


In [58]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel



In [59]:

diabetes= pd.read_csv('/content/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [60]:

X = diabetes.drop('Outcome', axis = 1)

y = diabetes["Outcome"]

In [61]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [62]:

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

sel = SelectFromModel(estimator=RandomForestClassifier(n_estimators=100))


fit = sel.fit(X_train, y_train)

In [63]:

sel_support = sel.get_support()
sel_support


array([False,  True, False, False, False,  True, False,  True])

In [64]:

selected_feat= X_train.loc[:,sel_support].columns.tolist()


print(str(len(selected_feat)), 'selected features')

3 selected features


In [65]:

print(selected_feat)

['Glucose', 'BMI', 'Age']


In [66]:

from sklearn.svm import LinearSVC

In [67]:


m = SelectFromModel(estimator=LinearSVC())


fit=m.fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [68]:

sel_support = m.get_support()
selected_feat= X.loc[:,sel_support].columns.tolist()

print(selected_feat)

['Pregnancies', 'DiabetesPedigreeFunction']


In [69]:


dia_df = pd.read_csv('/content/diabetes_cleaned.csv')
dia_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,218.937760,33.6,0.627,50.0,1
1,1.0,85.0,66.0,29.0,70.189298,26.6,0.351,31.0,0
2,8.0,183.0,64.0,29.0,269.968908,23.3,0.672,32.0,1
3,1.0,89.0,66.0,23.0,94.000000,28.1,0.167,21.0,0
4,0.0,137.0,40.0,35.0,168.000000,43.1,2.288,33.0,1


In [70]:

dia_df.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.539062,72.405184,29.108073,152.222767,32.307682,0.471876,33.240885,0.348958
std,3.369578,30.490660,12.096346,8.791221,97.387162,6.986674,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,-17.757186,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,25.000000,89.647494,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.202592,29.000000,130.000000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,188.448695,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [71]:
from sklearn import preprocessing

for i in dia_df:
    dia_df[[i]]=preprocessing.scale(dia_df[i].astype('float64'))


In [72]:

dia_df.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,2.544261e-17,-7.661695e-18,-4.474864e-16,-1.795800e-16,-2.428613e-17,9.880696e-17,2.612927e-16,1.857600e-16,2.408374e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-2.544700e+00,-4.004245e+00,-2.516429e+00,-1.746542e+00,-2.020543e+00,-1.189553e+00,-1.041549e+00,-7.321202e-01
25%,-8.448851e-01,-7.396938e-01,-6.953060e-01,-4.675972e-01,-6.429600e-01,-7.172147e-01,-6.889685e-01,-7.862862e-01,-7.321202e-01
50%,-2.509521e-01,-1.489643e-01,-1.675912e-02,-1.230129e-02,-2.283386e-01,-4.406715e-02,-3.001282e-01,-3.608474e-01,-7.321202e-01
75%,6.399473e-01,6.140612e-01,6.282695e-01,3.291706e-01,3.722209e-01,6.147581e-01,4.662269e-01,6.602056e-01,1.365896e+00
max,3.906578e+00,2.542136e+00,4.102655e+00,7.955377e+00,7.128551e+00,4.983056e+00,5.883565e+00,4.063716e+00,1.365896e+00


In [73]:

from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [74]:

X = dia_df.drop('Outcome', axis = 1)

y = dia_df["Outcome"]



X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [75]:

vif=pd.DataFrame()

 
vif['VIF Factor']=[variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

In [76]:


vif['Features']=X.columns

In [77]:

round(vif, 2)


,VIF Factor,Features
0,1.43,Pregnancies
1,2.07,Glucose
2,1.24,BloodPressure
3,1.43,SkinThickness
4,2.04,Insulin
5,1.58,BMI
6,1.05,DiabetesPedigreeFunction
7,1.62,Age


In [78]:


X= X.drop(['Glucose','Insulin','Age'], axis = 1)

In [79]:

vif=pd.DataFrame()


In [80]:

vif['VIF Factor']=[variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

vif['features']=X.columns
round(vif, 2)

,VIF Factor,features
0,1.05,Pregnancies
1,1.13,BloodPressure
2,1.42,SkinThickness
3,1.50,BMI
4,1.03,DiabetesPedigreeFunction


------------------------------